### Imports:

In [2]:
import wrangle

In [3]:
df = wrangle.get_geo_results(cached=True)
df = wrangle.
df.head()

,language,content
0,Python,Map Merger tool - tested using ArcMap 10.7\nWr...
1,Jupyter Notebook,wellio.js\nJavaScript for converting well-log ...
2,Python,geomodel-2-3dweb\n\nGenerates 3D web versions ...
3,JavaScript,GeoFeature\nGeological features of the Quanfoc...
4,JavaScript,U.S. Geological Survey Best Practices\nThis re...
